In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# import map matploylib graphinc and charting library
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib notebook

import folium # map rendering library

print('Libraries imported.')

Libraries imported.


# 1. Get St. Louis Neighborhood data

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_neighborhoods_of_St._Louis'
df_list = pd.read_html(url)

In [3]:
stl = df_list[1]

In [4]:
stl['Latitude'] = np.nan

stl['Longitude'] = np.nan


# 2. Geocode Neighborhoods

In [5]:
def getlatlong(Neighborhood):
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, St Louis, Missouri'.format(Neighborhood))
        lat_lng_coords = g.latlng
        print(Neighborhood, lat_lng_coords)
    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    return [latitude, longitude]

stl['Latitude'] = stl['Neighborhood'].apply(lambda x: getlatlong(x)[0])
stl['Longitude'] = stl['Neighborhood'].apply(lambda x: getlatlong(x)[1])

Academy [38.590748000000076, -90.50589499999995]
Baden [38.70553000000007, -90.23000999999994]
Benton Park [38.59962000000007, -90.21887999999996]
Benton Park West [38.597660000000076, -90.23096999999996]
Bevo Mill [38.58386000000007, -90.26637999999997]
Botanical Heights [38.620960000000025, -90.25062999999994]
Boulevard Heights [38.562110000000075, -90.27738999999997]
Carondelet [38.55639000000008, -90.26642999999996]
Carr Square [38.63901000000004, -90.19948999999997]
Central West End [38.641860000000065, -90.25025999999997]
Cheltenham [38.62736000000007, -90.27988999999997]
Clayton-Tamm [38.62704000000008, -90.29131999999998]
Clifton Heights [38.61140000000006, -90.29169999999993]
College Hill [38.674000000000035, -90.20877999999999]
Columbus Square [38.637140000000045, -90.18933999999996]
Compton Heights [38.61298000000005, -90.23625999999996]
DeBaliviere Place [38.649470000000065, -90.27808999999996]
Downtown [38.62549000000007, -90.19029999999998]
Downtown West [38.6297500000000

Visitation Park [38.65662000000003, -90.27560999999997]
Walnut Park East [38.69856000000004, -90.25052999999997]
Walnut Park West [38.70607000000007, -90.25645999999995]
Wells/Goodfellow [38.677410000000066, -90.27033999999998]
West End [38.66110000000003, -90.28764999999999]
Wydown/Skinker [38.63954000000007, -90.30426999999997]


In [6]:
stl[stl['Neighborhood'] == 'Academy']

,Neighborhood,Population,White,Black,Hispanic/Latino2,AIAN1,Asian,Mixed Race,Corridor,Latitude,Longitude
0,Academy,3006,16.9,54.7,20.5,1.52,4.3,3.5,North,38.590748,-90.505895


# Note, Geocoding of Academy is wrong
Should be 38.655556,-90.267778

In [7]:
stl.loc[stl['Neighborhood'] =='Academy', 'Latitude'] = 38.655556
stl.loc[stl['Neighborhood'] =='Academy', 'Longitude'] = -90.267778

In [28]:
stl.shape

(79, 11)

# 3. Plot neighborhood points to see results

In [9]:
# create map of St Louis using latitude and longitude values
map_stl = folium.Map(location=[38.6300,-90.26667], zoom_start=11)

# add markers to map
for lat, lng, label in zip(stl['Latitude'], stl['Longitude'], stl['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=2,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_stl)  
    
map_stl

# Show racial makeup of neighborhoods

In [10]:
stl_black = stl[stl['Black'] >= 70]
stl_white = stl[stl['White'] >= 70]
stl_mixed = stl[(stl['Black'] < 70) & (stl['White'] < 70)]

In [27]:
print(stl_black.shape)
print(stl_white.shape)
print(stl_mixed.shape)
stl['White'].median()

(31, 11)
(20, 11)
(28, 11)


35.6

In [11]:
## Now redraw the map 

# create map of St Louis using latitude and longitude values
map_race = folium.Map(location=[38.6300,-90.26667], zoom_start=11)
# add markers to map
for lat, lng, label in zip(stl_black['Latitude'], stl_black['Longitude'], stl_black['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='black',
        fill=True,
        fill_color='#000000',
        fill_opacity=0.7,
        parse_html=False).add_to(map_race)  
    
for lat, lng, label in zip(stl_white['Latitude'], stl_white['Longitude'], stl_white['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_race)      

for lat, lng, label in zip(stl_mixed['Latitude'], stl_mixed['Longitude'], stl_mixed['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='brown',
        fill=True,
        fill_color='#A52A2A',
        fill_opacity=0.7,
        parse_html=False).add_to(map_race)      

    
map_race

# 5. Use Foursquare to obtain venue information

In [12]:
CLIENT_ID = 'FBUSMB1E55HUZ4HCO5GMQKSOVBXDXEGACXAE4PPBDIS1DFSN' # your Foursquare ID
CLIENT_SECRET = 'X55DLMCA0J1NCGMUPOZW2QGBRJAYJUPFR2SYU20X5BLR3BB1' # your Foursquare Secret
ACCESS_TOKEN = 'MCTBZD2MR32PYRPMDWQMXWV1UKWKAMOG1WSUEOUL4UMGKZ0H' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 100
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: FBUSMB1E55HUZ4HCO5GMQKSOVBXDXEGACXAE4PPBDIS1DFSN
CLIENT_SECRET:X55DLMCA0J1NCGMUPOZW2QGBRJAYJUPFR2SYU20X5BLR3BB1


In [13]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [14]:
stl_venues = getNearbyVenues(names=stl['Neighborhood'],
                                   latitudes=stl['Latitude'],
                                   longitudes=stl['Longitude']
                                  )

Academy
Baden
Benton Park
Benton Park West
Bevo Mill
Botanical Heights
Boulevard Heights
Carondelet
Carr Square
Central West End
Cheltenham
Clayton-Tamm
Clifton Heights
College Hill
Columbus Square
Compton Heights
DeBaliviere Place
Downtown
Downtown West
Dutchtown
Ellendale
Fairground
Forest Park Southeast
Fountain Park
Fox Park
Franz Park
The Gate District
Grand Center
Gravois Park
Greater Ville
Hamilton Heights
The Hill
Hi-Pointe
Holly Hills
Hyde Park
JeffVanderLou
Kings Oak
Kingsway East
Kingsway West
Kosciusko
Lafayette Square
LaSalle Park
Lewis Place
Lindenwood Park
Marine Villa
Mark Twain
Mark Twain/I-70 Industrial
McKinley Heights
Midtown
Mount Pleasant
Near North Riverfront
North Hampton
North Point
North Riverfront
O’Fallon
Old North St. Louis
Patch
Peabody Darst Webbe
Penrose
Princeton Heights
Riverview
Shaw
Skinker-DeBaliviere
Soulard
South Hampton
Southwest Garden
St. Louis Hills
St. Louis Place
Tiffany
Tower Grove East
Tower Grove South
Vandeventer
The Ville
Visitation Par

In [15]:
stl_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Academy,38.655556,-90.267778,Third Degree Glass Factory,38.651953,-90.270579,Factory
1,Academy,38.655556,-90.267778,The Vino Gallery,38.649584,-90.262899,Art Gallery
2,Academy,38.655556,-90.267778,Dressel's Pub,38.649086,-90.260987,Gastropub
3,Academy,38.655556,-90.267778,Left Bank Books,38.648360,-90.260969,Bookstore
4,Academy,38.655556,-90.267778,The Improv Shop,38.650009,-90.260491,Comedy Club


In [16]:
print(stl_venues.shape)
stl_venue_count = stl_venues[['Neighborhood','Venue Category']].groupby('Neighborhood').count()
stl_unique_venues = stl_venues.drop_duplicates(subset=['Venue Latitude', 'Venue Longitude', 'Venue'], keep='last')
print(stl_unique_venues.shape)

(2948, 7)
(1851, 7)


In [17]:
# create map of St Louis using latitude and longitude values
map_venues = folium.Map(location=[38.6300,-90.26667], zoom_start=11)
# add markers to map
for lat, lng, label in zip(stl_unique_venues['Venue Latitude'], stl_unique_venues['Venue Longitude'], stl_unique_venues['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=1,
        popup=label,
        color='crimson',
        fill=False,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_venues)  
    
for lat, lng, label in zip(stl['Latitude'], stl['Longitude'], stl['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=2,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_venues)  

    
map_venues

# 6. Transform data

In [18]:
# one hot encoding
stl_onehot = pd.get_dummies(stl_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
stl_onehot['Neighborhood'] = stl_venues['Neighborhood']
# move neighborhood column to the first column
fixed_columns = [stl_onehot.columns[-1]] + list(stl_onehot.columns[:-1])
stl_onehot = stl_onehot[fixed_columns]

stl_onehot.head()

,Neighborhood,ATM,Accessories Store,Adult Boutique,Advertising Agency,American Restaurant,Antique Shop,Aquarium,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Workshop,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Burger Joint,Bus Station,Business Service,Cafeteria,Café,Cajun / Creole Restaurant,Candy Store,Casino,Check Cashing Service,Chinese Restaurant,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Basketball Court,College Bookstore,Comedy Club,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Disc Golf,Discount Store,Dive Bar,Dog Run,Donut Shop,Drugstore,Eastern European Restaurant,Electronics Store,Entertainment Service,Ethiopian Restaurant,Event Space,Exhibit,Factory,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Festival,Filipino Restaurant,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Football Stadium,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Travel,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,Health & Beauty Service,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hockey Field,Home Service,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Lawyer,Light Rail Station,Liquor Store,Locksmith,Lounge,Market,Martial Arts School,Massage Studio,Medical School,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music Store,Music Venue,National Park,New American Restaurant,Nightclub,Nightlife Spot,Non-Profit,Noodle House,Office,Opera House,Optical Shop,Other Repair Shop,Outdoor Sculpture,Outdoors & Recreation,Paper / Office Supplies Store,Park,Pawn Shop,Performing Arts Venue,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Photography Studio,Piano Bar,Picnic Shelter,Pie Shop,Pilates Studio,Pizza Place,Planetarium,Platform,Playground,Plaza,Poke Place,Pool,Print Shop,Pub,Public Art,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Resort,Restaurant,Rock Club,Roof Deck,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stables,Steakhouse,Storage Facility,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Thrift / Vintage Store,Track,Trail,Train,Train Station,Tram Station,Turkish Restaurant,Used Bookstore,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Volleyball Court,Waste Facility,Water Park,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,Academy,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [19]:
stl_onehot.shape

(2948, 269)

In [20]:
stl_grouped = stl_onehot.groupby('Neighborhood').sum().reset_index()
stl_grouped.head()

,Neighborhood,ATM,Accessories Store,Adult Boutique,Advertising Agency,American Restaurant,Antique Shop,Aquarium,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Workshop,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Burger Joint,Bus Station,Business Service,Cafeteria,Café,Cajun / Creole Restaurant,Candy Store,Casino,Check Cashing Service,Chinese Restaurant,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Basketball Court,College Bookstore,Comedy Club,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Disc Golf,Discount Store,Dive Bar,Dog Run,Donut Shop,Drugstore,Eastern European Restaurant,Electronics Store,Entertainment Service,Ethiopian Restaurant,Event Space,Exhibit,Factory,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Festival,Filipino Restaurant,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Football Stadium,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Travel,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,Health & Beauty Service,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hockey Field,Home Service,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Lawyer,Light Rail Station,Liquor Store,Locksmith,Lounge,Market,Martial Arts School,Massage Studio,Medical School,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music Store,Music Venue,National Park,New American Restaurant,Nightclub,Nightlife Spot,Non-Profit,Noodle House,Office,Opera House,Optical Shop,Other Repair Shop,Outdoor Sculpture,Outdoors & Recreation,Paper / Office Supplies Store,Park,Pawn Shop,Performing Arts Venue,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Photography Studio,Piano Bar,Picnic Shelter,Pie Shop,Pilates Studio,Pizza Place,Planetarium,Platform,Playground,Plaza,Poke Place,Pool,Print Shop,Pub,Public Art,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Resort,Restaurant,Rock Club,Roof Deck,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stables,Steakhouse,Storage Facility,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Thrift / Vintage Store,Track,Trail,Train,Train Station,Tram Station,Turkish Restaurant,Used Bookstore,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Volleyball Court,Waste Facility,Water Park,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,Academy,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,2,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,1,0,0,0,0,1,0,0,3,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,

# 7. Group grocery stores and merge venues with neighborhood data

In [21]:
stl_grouped['All Grocery'] = stl_grouped['Supermarket']+stl_grouped['Grocery Store']+ stl_grouped['Fruit & Vegetable Store'] + stl_grouped['Farmers Market']

In [53]:
stl_grocery = stl.merge(stl_grouped[['Neighborhood','All Grocery', 'Supermarket']], how='left', on='Neighborhood' )
stl_grocery.head()

,Neighborhood,Population,White,Black,Hispanic/Latino2,AIAN1,Asian,Mixed Race,Corridor,Latitude,Longitude,All Grocery,Supermarket
0,Academy,3006,16.9,54.7,20.5,1.52,4.3,3.5,North,38.655556,-90.267778,1,0
1,Baden,7268,6.3,91.8,0.5,0.10,0.0,1.3,North,38.705530,-90.230010,0,0
2,Benton Park,3532,68.2,25.1,3.2,0.30,1.2,3.8,South,38.599620,-90.218880,1,0
3,Benton Park West,4404,28.0,59.6,10.5,0.00,1.9,5.1,South,38.597660,-90.230970,3,0
4,Bevo Mill,12654,74.2,13.8,7.5,0.40,4.6,3.9,South,38.583860,-90.266380,1,0


In [56]:
gt = stl_grocery
gt['Group'] = ''
gt.loc[gt['Black'] >=84.9, 'Group'] = 'Black'
gt.loc[gt['White'] >=58.0, 'Group'] = 'White'
gt.loc[(gt['Black'] < 80.0) & (gt['White'] < 58.0), 'Group'] = 'Mixed'





,Neighborhood,Population,White,Black,Hispanic/Latino2,AIAN1,Asian,Mixed Race,Corridor,Latitude,Longitude,All Grocery,Supermarket,Group
0,Academy,3006,16.9,54.7,20.5,1.52,4.3,3.5,North,38.655556,-90.267778,1,0,Mixed
1,Baden,7268,6.3,91.8,0.5,0.10,0.0,1.3,North,38.705530,-90.230010,0,0,Black
2,Benton Park,3532,68.2,25.1,3.2,0.30,1.2,3.8,South,38.599620,-90.218880,1,0,White
3,Benton Park West,4404,28.0,59.6,10.5,0.00,1.9,5.1,South,38.597660,-90.230970,3,0,Mixed
4,Bevo Mill,12654,74.2,13.8,7.5,0.40,4.6,3.9,South,38.583860,-90.266380,1,0,White


In [83]:
gt[gt['Group'] == 'Mixed']['All Grocery'].describe()

count    25.000000
mean      1.320000
std       1.069268
min       0.000000
25%       1.000000
50%       1.000000
75%       2.000000
max       4.000000
Name: All Grocery, dtype: float64

In [88]:
import scipy.stats as stats

stats.f_oneway(#gt[gt['Group'] == 'Black']['All Grocery'],
               gt[gt['Group'] == 'Mixed']['All Grocery'],
               gt[gt['Group'] == 'White']['All Grocery']
              )

F_onewayResult(statistic=2.2181146025878, pvalue=0.14294123455434085)

In [89]:
stats.f_oneway(#gt[gt['Group'] == 'Black']['Supermarket'],
               gt[gt['Group'] == 'Mixed']['Supermarket'],
               gt[gt['Group'] == 'White']['Supermarket']
              )

F_onewayResult(statistic=0.16000000000000003, pvalue=0.6909310999464005)

In [ ]:
X = stl_grocery[['Supermarket','All Grocery']]
y = stl_grocery[['Race']]
sklearn.feature_selection.chi2(X, y)
#y.head()

In [ ]:
stl_ven_counts = stl.merge(stl_venue_count, how='left', on='Neighborhood' )

In [ ]:
BW_ven_counts = stl_ven_counts[['White','Black','Venue Category']].sort_values('White',axis=0, ascending=True, inplace=False)
BW_ven_counts

In [ ]:

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')


# For each set of style and range settings, plot n random points in the box
# defined by x in [23, 32], y in [0, 100], z in [zlow, zhigh].

xs = BW_ven_counts['White']
ys = BW_ven_counts['Black']
zs = BW_ven_counts['Venue Category']
ax.scatter(xs, ys, zs, marker=".", s=30)

ax.set_xlabel('Percent White')
ax.set_ylabel('Percent Black')
ax.set_zlabel('Number of venues')

plt.show()

In [ ]:
ven_black = BW_ven_counts[BW_ven_counts['Black'] >= 70]
ven_white = BW_ven_counts[BW_ven_counts['White'] >= 70]
ven_mixed = BW_ven_counts[(BW_ven_counts['Black'] < 70) & (BW_ven_counts['White'] < 70)]

In [ ]:
data = [ven_black['Venue Category'], ven_mixed['Venue Category'], ven_white['Venue Category']]
fig7, ax7 = plt.subplots()
ax7.set_title('Numbers of Venues by Racial Makeup')

ax7.boxplot(data)
plt.xticks([1, 2, 3], ['Black>70','Mixed','White>70'])
plt.show()

# 8. Cluster neighborhoods by most frequent businesses

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = stl_grouped['Neighborhood']

for ind in np.arange(stl_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(stl_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

In [ ]:
# set number of clusters
kclusters = 5

stl_grouped_clustering = stl_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(stl_grouped_clustering)



In [ ]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

stl_clustered = stl

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
stl_clustered = stl_clustered.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')



stl_clustered['Cluster Labels'].value_counts()

In [ ]:
# create map
map_clusters = folium.Map(location=[38.6300,-90.26667], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(stl_clustered['Latitude'], stl_clustered['Longitude'], stl_clustered['Neighborhood'], stl_clustered['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [ ]:
stl_merged = stl_clustered.merge(stl_grocery[['Neighborhood','Supermarket', 'All Grocery']], how='left', on='Neighborhood' )

In [ ]:
stl_merged.groupby('Cluster Labels').mean().reset_index()

In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Cluster Labels']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
clusters_venues_sorted = pd.DataFrame(columns=columns)
clusters_venues_sorted['Cluster Labels'] = stl_grp_clus['Cluster Labels']

for ind in np.arange(stl_grp_clus.shape[0]):
    clusters_venues_sorted.iloc[ind, 1:] = return_most_common_venues(stl_grp_clus.iloc[ind, :], num_top_venues)

clusters_venues_sorted.head()

In [ ]:
stl_grocery[stl_grocery['Black'] > 90] 